## Importamos librerias necesarias para iniciar EDA

In [121]:
import pandas as pd
import numpy as np
import warnings
import re
import json
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

# Text
import nltk
nltk.download('punk')
nltk.download('stopwords')

from nltk import regexp_tokenize


warnings.filterwarnings('ignore')#to filter all the warnings
pd.set_option('float_format', '{:.4f}'.format)# to keep the float values short

[nltk_data] Error loading punk: Package 'punk' not found in index
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/kennydev/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Obtenemos la data necesaria

1. Youtube Trending Statistics `(CSV)` - *Todo*

In [68]:
df_youtube = pd.read_csv('../data/raw/USvideos.csv')
df_youtube.head()

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description
0,2kyS6SvSYSE,17.14.11,WE WANT TO TALK ABOUT OUR MARRIAGE,CaseyNeistat,22,2017-11-13T17:13:01.000Z,SHANtell martin,748374,57527,2966,15954,https://i.ytimg.com/vi/2kyS6SvSYSE/default.jpg,False,False,False,SHANTELL'S CHANNEL - https://www.youtube.com/s...
1,1ZAPwfrtAFY,17.14.11,The Trump Presidency: Last Week Tonight with J...,LastWeekTonight,24,2017-11-13T07:30:00.000Z,"last week tonight trump presidency|""last week ...",2418783,97185,6146,12703,https://i.ytimg.com/vi/1ZAPwfrtAFY/default.jpg,False,False,False,"One year after the presidential election, John..."
2,5qpjK5DgCt4,17.14.11,"Racist Superman | Rudy Mancuso, King Bach & Le...",Rudy Mancuso,23,2017-11-12T19:05:24.000Z,"racist superman|""rudy""|""mancuso""|""king""|""bach""...",3191434,146033,5339,8181,https://i.ytimg.com/vi/5qpjK5DgCt4/default.jpg,False,False,False,WATCH MY PREVIOUS VIDEO ▶ \n\nSUBSCRIBE ► http...
3,puqaWrEC7tY,17.14.11,Nickelback Lyrics: Real or Fake?,Good Mythical Morning,24,2017-11-13T11:00:04.000Z,"rhett and link|""gmm""|""good mythical morning""|""...",343168,10172,666,2146,https://i.ytimg.com/vi/puqaWrEC7tY/default.jpg,False,False,False,Today we find out if Link is a Nickelback amat...
4,d380meD0W0M,17.14.11,I Dare You: GOING BALD!?,nigahiga,24,2017-11-12T18:01:41.000Z,"ryan|""higa""|""higatv""|""nigahiga""|""i dare you""|""...",2095731,132235,1989,17518,https://i.ytimg.com/vi/d380meD0W0M/default.jpg,False,False,False,I know it's been a while since we did this sho...


2. Obtenemos la información contenida en el archivo `JSON
`

In [69]:
with open('../data/raw/US_category_id.json', 'r') as f:
    category_us = json.loads(f.read())

df_us_cat = pd.json_normalize(category_us, record_path='items')
df_us_cat.rename({'id':'category_id'}, axis=1, inplace=True)
df_us_cat['category_id'] = df_us_cat['category_id'].astype('int64')
df_us_cat.head(3)

,kind,etag,category_id,snippet.channelId,snippet.title,snippet.assignable
0,youtube#videoCategory,"""m2yskBQFythfE4irbTIeOgYYfBU/Xy1mB4_yLrHy_BmKm...",1,UCBR8-60-B28hp2BmDPdntcQ,Film & Animation,True
1,youtube#videoCategory,"""m2yskBQFythfE4irbTIeOgYYfBU/UZ1oLIIz2dxIhO45Z...",2,UCBR8-60-B28hp2BmDPdntcQ,Autos & Vehicles,True
2,youtube#videoCategory,"""m2yskBQFythfE4irbTIeOgYYfBU/nqRIq97-xe5XRZTxb...",10,UCBR8-60-B28hp2BmDPdntcQ,Music,True


3. Para tener toda la información en un solo `DataFrame` realizamos un merge con los datasets creados anteriormente **df_us_cat** y **df_youtube** 

In [70]:
df = pd.merge(df_youtube, df_us_cat, how='left', on='category_id')
df.head(1)

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,...,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description,kind,etag,snippet.channelId,snippet.title,snippet.assignable
0,2kyS6SvSYSE,17.14.11,WE WANT TO TALK ABOUT OUR MARRIAGE,CaseyNeistat,22,2017-11-13T17:13:01.000Z,SHANtell martin,748374,57527,2966,...,https://i.ytimg.com/vi/2kyS6SvSYSE/default.jpg,False,False,False,SHANTELL'S CHANNEL - https://www.youtube.com/s...,youtube#videoCategory,"""m2yskBQFythfE4irbTIeOgYYfBU/xId8RX7vRN8rqkbYZ...",UCBR8-60-B28hp2BmDPdntcQ,People & Blogs,True


## Limpieza de los datos

1. Existen columnas que no aportan información relevante a nuestro EDA, algunas de ellas son:
'thumbnail_link','kind','etag','id','snippet.assignable','snippet.channelId','channelId'

* thumbnail_link : imagenes en miniatura
* kind : columna con texto irrelevante
* etad : igual que kind 

In [71]:
columns_drop = ['thumbnail_link','kind','etag','snippet.assignable','snippet.channelId']
df = df.drop(columns_drop, axis=1)
print("DataFrame post cleaned shape: {}".format(df.shape))

DataFrame post cleaned shape: (40949, 16)


2. Observemos el comportamiento y coherencia de los datos

**Premisas**
* Un video que no tenga visitas, no puede tener likes ni dislikes
* La cantidad de likes o dislike, no puede ser mayor a la cantidad de visitas
* Un video con las reacciones deshabilitadas, no puede tener like ni dislikes
* Un video con comentarios deshabilitados, su cantidad de comentarios debe ser cero

In [72]:
df.sample(3)

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,comments_disabled,ratings_disabled,video_error_or_removed,description,snippet.title
10632,D0aAYfRBVik,18.06.01,Katy Perry - Making Of “Hey Hey Hey” Music Video,KatyPerryVEVO,10,2018-01-04T18:00:00.000Z,"katy perry|""hey hey hey""|""behind the scenes""|""...",1055667,50532,2048,2519,False,False,False,Get Hey Hey Hey from Katy's new album 'Witness...,Music
12775,PGH1V_yZq7k,18.18.01,"Helen Mirren, Donald Sutherland Talk Oscars Ho...",The View,24,2018-01-12T16:57:13.000Z,"helen mirren|""donald sutherland""|""the view""|""h...",150617,1315,395,247,False,False,False,NaN,Entertainment
18804,D_eZxSYRhco,18.18.02,READY PLAYER ONE - Come With Me,Warner Bros. Pictures,24,2018-02-15T17:00:02.000Z,"ready player one|""ready player one movie""|""rpo...",2515101,27796,1042,5757,False,False,False,http://readyplayeronemovie.com\nhttp://faceboo...,Entertainment


In [73]:
# Video sin visitas, no tiene likes ni dislikes
df[(df['views'] == 0) & ((df['likes'] > 0) | (df['dislikes'] > 0))]

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,comments_disabled,ratings_disabled,video_error_or_removed,description,snippet.title


In [74]:
df.describe()

,category_id,views,likes,dislikes,comment_count
count,40949.0000,40949.0000,40949.0000,40949.0000,40949.0000
mean,19.9724,2360784.6383,74266.7024,3711.4009,8446.8037
std,7.5683,7394113.7597,228885.3382,29029.7059,37430.4870
min,1.0000,549.0000,0.0000,0.0000,0.0000
25%,17.0000,242329.0000,5424.0000,202.0000,614.0000
50%,24.0000,681861.0000,18091.0000,631.0000,1856.0000
75%,25.0000,1823157.0000,55417.0000,1938.0000,5755.0000
max,43.0000,225211923.0000,5613827.0000,1674420.0000,1361580.0000


No existen videos con un numero de vistas igual o menor a cero, lo que guarda relación con la cantidad de likes y dislikes

In [75]:
# Cantidad de likes o dislikes, no puede ser mayor a cantidad de views
df[(df['likes'] > df['views']) | (df['dislikes'] > df['views'])].sum()

video_id                 0.0000
trending_date            0.0000
title                    0.0000
channel_title            0.0000
category_id              0.0000
publish_time             0.0000
tags                     0.0000
views                    0.0000
likes                    0.0000
dislikes                 0.0000
comment_count            0.0000
comments_disabled        0.0000
ratings_disabled         0.0000
video_error_or_removed   0.0000
description              0.0000
snippet.title            0.0000
dtype: float64

No existen videos que cuenten con mayor cantidad de likes o dislikes que vistas

In [76]:
# Videos con reacciones deshabilitadas, no puede tener likes ni dislikes
df[(df['ratings_disabled'] == True) & (df['likes'] > 0)].sum()

video_id                 0.0000
trending_date            0.0000
title                    0.0000
channel_title            0.0000
category_id              0.0000
publish_time             0.0000
tags                     0.0000
views                    0.0000
likes                    0.0000
dislikes                 0.0000
comment_count            0.0000
comments_disabled        0.0000
ratings_disabled         0.0000
video_error_or_removed   0.0000
description              0.0000
snippet.title            0.0000
dtype: float64

In [77]:
df[(df['ratings_disabled'] == True) & (df['dislikes'] > 0)].sum()

video_id                 0.0000
trending_date            0.0000
title                    0.0000
channel_title            0.0000
category_id              0.0000
publish_time             0.0000
tags                     0.0000
views                    0.0000
likes                    0.0000
dislikes                 0.0000
comment_count            0.0000
comments_disabled        0.0000
ratings_disabled         0.0000
video_error_or_removed   0.0000
description              0.0000
snippet.title            0.0000
dtype: float64

Los videos con las reacciones deshabilitads, no cuentan con likes o dislikes

In [78]:
# Videos con comentarios deshabilitados, no deben contar con comentarios
df[(df['comment_count'] > 0) & (df['comments_disabled'] == True)].count()

video_id                  0
trending_date             0
title                     0
channel_title             0
category_id               0
publish_time              0
tags                      0
views                     0
likes                     0
dislikes                  0
comment_count             0
comments_disabled         0
ratings_disabled          0
video_error_or_removed    0
description               0
snippet.title             0
dtype: int64

Los videos con los comentarios deshabilitades, no cuentan con ningún comentario

<p style='color: green'>Conclusion</p>

La salud de las columnas númericas, y su coherencia parecen ser correctas, de acuerdo a las premisas analizadas anteriormente, por ello podemos hacer drop de las columnas `comments_disabled` y `ratings_disabled` ya que esta información puede ser interpretada de sus derivados `likes`, `dislikes` y `comment_count`

In [79]:
df = df.drop(['comments_disabled', 'ratings_disabled'], axis=1)

3. Analizaremos datos faltantes y duplicados en nuestro dataset

In [80]:
print("""
      ****************************************
            COLUMNAS CON DATOS NULOS
      ****************************************""")
print(df.isna().sum())




      ****************************************
            COLUMNAS CON DATOS NULOS
      ****************************************
video_id                    0
trending_date               0
title                       0
channel_title               0
category_id                 0
publish_time                0
tags                        0
views                       0
likes                       0
dislikes                    0
comment_count               0
video_error_or_removed      0
description               570
snippet.title               0
dtype: int64


Tenemos 570 registros `NaN` en columna descripcion, observaremos algunas filas con estos valores nulos para tomar una decisión sobre como lidiar con ellos

In [81]:
df[df['description'].isna()].sample(10)

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,video_error_or_removed,description,snippet.title
1599,aujUl3yt6nM,17.21.11,Quad9 How To Install with Windows,Quad9 DNS,22,2017-11-16T01:56:43.000Z,"DNS|""privacy""|""security""",5963,47,2,13,False,NaN,People & Blogs
14360,Cd2Ue_xw1oo,18.26.01,John Varvatos Spring/Summer 2018 Campaign Feat...,John Varvatos,24,2018-01-22T02:28:45.000Z,[none],4131,121,1,8,False,NaN,Entertainment
35679,Hzk1bM2vVFU,18.19.05,True Facts : Carnivorous Plants,zefrank1,22,2018-05-07T19:50:01.000Z,[none],1425300,83921,765,5693,False,NaN,People & Blogs
18871,oTKbyqkAu7s,18.18.02,Senator Murphy on the South Florida School Sho...,Senator Chris Murphy,25,2018-02-14T20:27:58.000Z,[none],50735,807,488,1022,False,NaN,News & Politics
19298,iveV0Q77gvU,18.20.02,Catt Sadler Shares Her Side Of E! Exit | The View,The View,24,2018-02-14T16:14:44.000Z,"Catt Sadler|""The View""|""E! news""|""hot topics""|...",183721,2163,297,740,False,NaN,Entertainment
15980,kZete48ZtsY,18.03.02,Deleted video,DaHoopSpot Productions,17,2017-12-16T04:29:25.000Z,[none],2620,8,12,5,True,NaN,Sports
23034,uiZprtMUyCM,18.11.03,'Bachelor' Finale: Worst Breakup Ever? | The View,The View,24,2018-03-06T16:15:15.000Z,"The Bachelor|""The View""|""hot topics""|""entertai...",370987,2002,1231,785,False,NaN,Entertainment
34493,zxwfDlhJIpw,18.13.05,kanye west / charlamagne interview,Kanye West,22,2018-05-01T15:57:06.000Z,"Kanye West|""YEEZY""|""Kanye""|""Charlamagne""|""The ...",8215762,164279,19213,47898,False,NaN,People & Blogs
1852,kKFNThEiilU,17.23.11,Voices,Doug Jones for Senate,22,2017-11-21T02:30:47.000Z,[none],84038,823,340,486,False,NaN,People & Blogs
22913,lRlp6e2Bam8,18.10.03,Delaware Linemen Help Get The Power On in Puer...,N W,22,2018-03-01T00:43:04.000Z,[none],299149,5628,249,0,False,NaN,People & Blogs


No existe un patron definido, que nos indique la razón de la ausencia de descripciones en los videos del dataset, así que llenaremos estos valores nulos con `strings` vacios

In [82]:
df['description'].fillna('', inplace=True)

### Tags, Channel Title, Video Title y Descripciones con caracteres validos
* Strings que puedan ser decodificados bajo el protocolo ASCII - Columna `descriptions`

In [83]:
# Encuentra cadenas de caracteres que pueden o no ser decodificados por el protocolo ASCII
def is_english_string(text):
    try: 
        text.encode('ascii') 
    except UnicodeEncodeError: 
        return False
    else: 
        return True


In [84]:
# Remueve palabras con caracteres especiales y retorna una nueva cadena sin esas palabras

def remove_words_especial_char(text):
    new_text = []
    for word in text.split():
        if is_english_string(word):
            new_text.append(word)
    
    return " ".join(new_text)

In [85]:
df['des_is_english'] = df['description'].apply(lambda d: is_english_string(d))

In [86]:
df['cln_description'] = df['description'].apply(lambda d: remove_words_especial_char(d))

In [87]:
df[df['des_is_english'] == False][['cln_description', 'description']]

,cln_description,description
2,WATCH MY PREVIOUS VIDEO \n\nSUBSCRIBE https://...,WATCH MY PREVIOUS VIDEO ▶ \n\nSUBSCRIBE ► http...
3,Today we find out if Link is a Nickelback amat...,Today we find out if Link is a Nickelback amat...
5,Using the iPhone for the past two weeks -- her...,Using the iPhone for the past two weeks -- her...
7,Ice Cream Pint Combination Lock - http://amzn....,Ice Cream Pint Combination Lock - http://amzn....
12,WATCH SILENCE MUSIC VIDEO https://youtu.be/Tx1...,WATCH SILENCE MUSIC VIDEO ▶ https://youtu.be/T...
...,...,...
40935,Chun-Li (Live on SNL / 2018)Song Available Her...,Chun-Li (Live on SNL / 2018)Song Available Her...
40938,Daddy Yankee - Hielo (Video Oficial)Spotify: h...,Daddy Yankee - Hielo (Video Oficial)Spotify: h...
40941,Stream + Download:https://wmna.sh/speechlessht...,Stream + Download:https://wmna.sh/speechlessht...
40946,I had so much fun transforming Safiyas hair in...,I had so much fun transforming Safiyas hair in...


Terminado el procedimiento podemos hacer drop sobre las columnas temporales, creadas para esta limpieza

In [88]:
df.drop(['des_is_english', 'description'], axis=1, inplace=True)

* Strings que puedan ser decodificados bajo el protocolo ASCII - Columna `tags`

In [89]:
df['tags_is_english'] = df['tags'].apply(lambda t: is_english_string(t))

In [90]:
df['cln_tags'] = df['tags'].apply(lambda d: remove_words_especial_char(d))

In [91]:
df[df['tags_is_english'] == False][['cln_tags', 'tags']]

,cln_tags,tags
52,"wwe|""world wrestling Top 10""|""Stone Cold Steve...","wwe|""world wrestling entertainment""|""wrestling..."
82,"cave|""cheese""|""goat""|""rind""|""sheep's milk""|""cr...","cave|""cheese""|""goat""|""rind""|""sheep's milk""|""cr..."
88,"Alan Walker|""DJ Walkzz""|""K-391""|""House""|""Techn...","Alan Walker|""DJ Walkzz""|""K-391""|""House""|""Techn..."
100,"Japan|""Japanese""|""yt:cc=on""|""idiot's""|""guide""|...","Japan|""Japanese""|""yt:cc=on""|""idiot's""|""guide""|..."
128,"cartoon|""simons cat""|""simon's cat""|""simonscat""...","cartoon|""simons cat""|""simon's cat""|""simonscat""..."
...,...,...
40874,"Let's Lets Go""|""Pokemon Lets Go""|""Pokemon Let'...","Pokémon|""Pokemon""|""Pokémon Let's Go""|""Pokémon ..."
40887,,"cook|""cat""|""Jun""|""Rachel""|""Japanese""|""food""|""j..."
40893,"cartoon|""simons cat""|""simon's cat""|""simonscat""...","cartoon|""simons cat""|""simon's cat""|""simonscat""..."
40896,,"방탄소년단|""BTS""|""BANGTAN""|""HIPHOP""|""랩몬스터""|""RapMons..."


In [92]:
df.drop(['tags_is_english', 'tags'], axis=1, inplace=True)

* Strings que puedan ser decodificados bajo el protocolo ASCII - Columna `title`

In [93]:
df['title_is_english'] = df['title'].apply(lambda t: is_english_string(t))
df['cln_title'] = df['title'].apply(lambda d: remove_words_especial_char(d))
df[df['title_is_english'] == False][['cln_title', 'title']]

,cln_title,title
9,Why the rise of the robots mean the end of work,Why the rise of the robots won’t mean the end ...
32,Eminem - Walk On Water (Audio) ft.,Eminem - Walk On Water (Audio) ft. Beyoncé
57,Kellyanne Conway on Roy Moore This Week Abc: N...,Kellyanne Conway on Roy Moore This Week Abc: T...
68,Watch Norman Reedus Come Face to Face with his...,Watch Norman Reedus Come Face to Face with his...
75,Rosie On Donald Hostility Toward Her | WWHL,Rosie O’Donnell On Donald Trump’s Hostility To...
...,...,...
40874,"Let's Go, Pikachu! and Let's Go, Eevee! Trailer","Pokémon: Let's Go, Pikachu! and Pokémon: Let's..."
40896,[CHOREOGRAPHY] BTS 'FAKE LOVE' Dance Practice,[CHOREOGRAPHY] BTS (방탄소년단) 'FAKE LOVE' Dance P...
40910,HONNE - Location Unknown (feat. Georgia),HONNE - Location Unknown ◐ (feat. Georgia)
40923,Camels vs. Cactus!!!,Camels vs. Cactus!!! جمل


In [94]:
df.drop(['title_is_english', 'title'], axis=1, inplace=True)

* Strings que puedan ser decodificados bajo el protocolo ASCII - Columna `channel_title`

In [95]:
df['chnl_title_is_english'] = df['channel_title'].apply(lambda t: is_english_string(t))
df['cln_channel_title'] = df['channel_title'].apply(lambda d: remove_words_especial_char(d))
df[df['chnl_title_is_english'] == False][['cln_channel_title', 'channel_title']]

,cln_channel_title,channel_title
47,NBA Highlights YouTube,NBA Highlights · YouTube
82,Bon,Bon Appétit
296,Lalonde,Estée Lalonde
312,NBA Highlights YouTube,NBA Highlights · YouTube
359,Bon,Bon Appétit
...,...,...
40464,The Official YouTube channel,The Official Pokémon YouTube channel
40615,Bon,Bon Appétit
40668,The Official YouTube channel,The Official Pokémon YouTube channel
40764,Bon,Bon Appétit


In [96]:
df.drop(['chnl_title_is_english', 'channel_title'], axis=1, inplace=True)

Observemos nuestro dataset ahora

In [97]:
df.head(3)

,video_id,trending_date,category_id,publish_time,views,likes,dislikes,comment_count,video_error_or_removed,snippet.title,cln_description,cln_tags,cln_title,cln_channel_title
0,2kyS6SvSYSE,17.14.11,22,2017-11-13T17:13:01.000Z,748374,57527,2966,15954,False,People & Blogs,SHANTELL'S CHANNEL - https://www.youtube.com/s...,SHANtell martin,WE WANT TO TALK ABOUT OUR MARRIAGE,CaseyNeistat
1,1ZAPwfrtAFY,17.14.11,24,2017-11-13T07:30:00.000Z,2418783,97185,6146,12703,False,Entertainment,"One year after the presidential election, John...","last week tonight trump presidency|""last week ...",The Trump Presidency: Last Week Tonight with J...,LastWeekTonight
2,5qpjK5DgCt4,17.14.11,23,2017-11-12T19:05:24.000Z,3191434,146033,5339,8181,False,Comedy,WATCH MY PREVIOUS VIDEO \n\nSUBSCRIBE https://...,"racist superman|""rudy""|""mancuso""|""king""|""bach""...","Racist Superman | Rudy Mancuso, King Bach & Le...",Rudy Mancuso


## Reemplazo de caracteres especiales sobre columnas tipo object
* Deseamos dejar el modelo completamente preprocesado antes de crear cualquier modelo(optimizacion) u obtener insights

<p style='color: yellow'>Opciones</p>

* Usar la libreria `re` y con ella expresiones regulares
* Utilizar el modulo `str` incluido en python y con el la función `replace`


In [98]:
flex_pattern = r'''(?x)                  # Flag para iniciar el modo verbose
              (?:[A-Z]\.)+            # Hace match con abreviaciones como U.S.A.
              | \w+(?:-\w+)*         # Hace match con palabras que pueden tener un guión interno
              | \$?\d+(?:\.\d+)?%?  # Hace match con dinero o porcentajes como $15.5 o 100%
              | \.\.\.              # Hace match con puntos suspensivos
              | [][.,;'?"():-_`]    # Hace match con signos de puntuación
              
              '''

pattern = r'''(?x)                  # Flag para iniciar el modo verbose
              (?:[A-Z]\.)+            # Hace match con abreviaciones como U.S.A.
              | \w+(?:-\w+)*         # Hace match con palabras que pueden tener un guión interno
              | \$?\d+(?:\.\d+)?%?  # Hace match con dinero o porcentajes como $15.5 o 100%
              | \.\.\.              # Hace match con puntos suspensivos
              
              '''

In [99]:
def clean_words(text, pattern):
    tokens = regexp_tokenize(text, pattern)
    return ' '.join(tokens)

1. Usando expresiones regulares sobre la columna `cln_tags`

In [100]:
df['post_cln_tags'] = df['cln_tags'].apply(lambda t: clean_words(t, pattern))
df['post_cln_tags'] = df['post_cln_tags'].apply(lambda t: t.lower())
df[['post_cln_tags', 'cln_tags']].sample(10)

,post_cln_tags,cln_tags
14014,god of war kratos atreus e3 2017 e3 gameplay c...,"God of War|""Kratos""|""Atreus""|""E3 2017""|""E3""|""G..."
33475,i let my subscribers pick my makeup subscriber...,"i let my subscribers pick my makeup|""subscribe..."
39844,none,[none]
29490,insider realistic fake food fake food artists ...,"INSIDER|""realistic fake food""|""fake food artis..."
38885,meghan markle prince harry royal wedding megha...,"meghan markle|""prince harry""|""royal wedding""|""..."
7237,beauty chaka khan hair how to how-to makeup ti...,"beauty|""chaka khan""|""hair""|""how to""|""how-to""|""..."
38725,rita ora rita ora your song anywhere official ...,"Rita Ora|""Rita""|""Ora""|""Your Song""|""Anywhere""|""..."
14811,mamma mia mamma mia 2 mamma mia movie mamma mi...,"Mamma Mia|""Mamma Mia 2""|""Mamma Mia Movie""|""Mam..."
13179,hulu,hulu
23003,ladylike women fashion desk makeover disney bu...,"Ladylike|""Women""|""Fashion""|""Desk makeover""|""Di..."


2. Usando expresiones regulares sobre la columna `cln_title`

In [101]:
df['post_cln_title'] = df['cln_title'].apply(lambda t: clean_words(t, pattern))
df['post_cln_title'] = df['post_cln_title'].apply(lambda t: t.lower())
df[['post_cln_title', 'cln_title']].sample(10)

,post_cln_title,cln_title
15784,being a sore loser anwar jibawi,Being a Sore Loser | Anwar Jibawi
26987,leslie odom jr learned the secret to success t...,Leslie Odom Jr. Learned The Secret To Success:...
13009,jamie dornan had an awkward moment the graham ...,Jamie Dornan Had an Awkward Moment | The Graha...
13290,ll sound in welsh,LL sound in Welsh
16117,investigating concussions in the n.f.l. the tr...,Investigating Concussions in the N.F.L. | The ...
6326,spoilers mid-season finale talked about scene ...,(SPOILERS) Mid-Season Finale Talked About Scen...
24947,primitive technology lime,Primitive Technology: Lime
11881,controlling swarms of flies and gnats by singi...,Controlling swarms of flies and gnats by singi...
26484,4 producers flip the same sample,4 PRODUCERS FLIP THE SAME SAMPLE
20629,ed sheeran supermarket flowers live from the b...,Ed Sheeran - Supermarket Flowers [Live from th...


3. Usando expresiones regulares sobre la columna `cln_channel_title`

In [102]:
df['post_cln_channel_title'] = df['cln_channel_title'].apply(lambda t: clean_words(t, pattern))
df['post_cln_channel_title'] = df['post_cln_channel_title'].apply(lambda t: t.lower())
df[['post_cln_channel_title', 'cln_channel_title']].sample(10)

,post_cln_channel_title,cln_channel_title
40786,michael dapaah,Michael Dapaah
25089,ign,IGN
20048,gus johnson,Gus Johnson
40270,matthewsantoro,MatthewSantoro
30118,calvinharrisvevo,CalvinHarrisVEVO
4963,the food ranger,The Food Ranger
38027,bbcradio1vevo,BBCRadio1VEVO
17449,aunty donna,Aunty Donna
14228,msnbc,MSNBC
15024,saturday night live,Saturday Night Live


In [103]:
df.columns

Index(['video_id', 'trending_date', 'category_id', 'publish_time', 'views',
       'likes', 'dislikes', 'comment_count', 'video_error_or_removed',
       'snippet.title', 'cln_description', 'cln_tags', 'cln_title',
       'cln_channel_title', 'post_cln_tags', 'post_cln_title',
       'post_cln_channel_title'],
      dtype='object')

4. Usando expresiones regulares sobre la columna `cln_description`

In [104]:
df['post_cln_description'] = df['cln_description'].apply(lambda t: clean_words(t, pattern))
df['post_cln_description'] = df['post_cln_description'].apply(lambda t: t.lower())
df[['post_cln_description', 'cln_description']].sample(10)

,post_cln_description,cln_description
21122,every journey has a destination every song has...,Every journey has a destination. Every song ha...
28402,to kevin hart not partner human coffee tables ...,"To Kevin Hart, not partner human coffee tables..."
25476,don t doubt the streetwear nasa fanny pack it ...,Don't doubt the streetwear NASA fanny pack it'...
6656,how cool would it be to have a retro arcade ga...,How cool would it be to have a retro arcade ga...
32929,is having high-end home decor important to you...,Is having high-end home decor important to you...
27749,spoiler alert there s still a bixby button n n...,Spoiler alert: There's still a Bixby button.\n...
31084,stream download queendom https lnk to aqu1id i...,Stream/Download Queendom: https://lnk.to/AQu1I...
29564,amy schumer talked with ellen about getting en...,Amy Schumer talked with Ellen about getting en...
39041,download war robots 6v6 pvp game and get aweso...,Download War Robots 6v6 PvP game and get AWESO...
10823,funny videos try not to laugh smile or grin wh...,"Funny videos try not to laugh, smile or grin w..."


Una vez terminado esta etapa, podemos eliminar las columnas que sufrieron el preprocesamiento y renombrar las persistentes

In [105]:
df.columns
columnas_repetidas = ['cln_description', 'cln_tags', 'cln_title', 'cln_channel_title', 'video_error_or_removed']
df.drop(columnas_repetidas, axis=1, inplace=True)

In [109]:
df.rename({'snippet.title':'category_name', 
           'post_cln_tags':'tags',
           'post_cln_title':'video_title',
           'post_cln_channel_title':'channel_title',
           'post_cln_description':'descrption'}, axis=1, inplace=True)
df.head(2)

,video_id,trending_date,category_id,publish_time,views,likes,dislikes,comment_count,category_name,tags,video_title,channel_title,descrption
0,2kyS6SvSYSE,17.14.11,22,2017-11-13T17:13:01.000Z,748374,57527,2966,15954,People & Blogs,shantell martin,we want to talk about our marriage,caseyneistat,shantell s channel https www youtube com shant...
1,1ZAPwfrtAFY,17.14.11,24,2017-11-13T07:30:00.000Z,2418783,97185,6146,12703,Entertainment,last week tonight trump presidency last week t...,the trump presidency last week tonight with jo...,lastweektonight,one year after the presidential election john ...


In [114]:
df.tail(2)

,video_id,trending_date,category_id,publish_time,views,likes,dislikes,comment_count,category_name,tags,video_title,channel_title,descrption
40947,oV0zkMe1K8s,18.14.06,1,2018-05-17T17:00:04.000Z,5660813,192957,2846,13088,Film & Animation,black panther hishe marvel infinity war how it...,how black panther should have ended,how it should have ended,how black panther should have endedwatch more ...
40948,ooyjaVdt-jA,18.14.06,20,2018-05-17T17:09:38.000Z,10306119,357079,212976,144795,Gaming,call of duty cod activision black ops 4,official call of black ops 4 multiplayer revea...,call of duty,call of duty black ops 4 multiplayer raises th...


## Tipos de datos adecuados

* Se detectaron columnas que contienen fechas con dtype object
* La columna `category_name` puede tener tipo `category`

In [111]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40949 entries, 0 to 40948
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   video_id       40949 non-null  object
 1   trending_date  40949 non-null  object
 2   category_id    40949 non-null  int64 
 3   publish_time   40949 non-null  object
 4   views          40949 non-null  int64 
 5   likes          40949 non-null  int64 
 6   dislikes       40949 non-null  int64 
 7   comment_count  40949 non-null  int64 
 8   category_name  40949 non-null  object
 9   tags           40949 non-null  object
 10  video_title    40949 non-null  object
 11  channel_title  40949 non-null  object
 12  descrption     40949 non-null  object
dtypes: int64(5), object(8)
memory usage: 5.4+ MB


In [125]:
def convert_str_to_datetime(string):
    date_time_obj = datetime.strptime(string, '%y.%d.%m')
    return date_time_obj

In [129]:
df['trending_date'] = df['trending_date'].apply(lambda date: convert_str_to_datetime(date))
df['publish_time'] = pd.to_datetime(df['publish_time']).dt.tz_convert(None)
df['category_name']= df['category_name'].astype('category')

In [130]:
df.head(2)

,video_id,trending_date,category_id,publish_time,views,likes,dislikes,comment_count,category_name,tags,video_title,channel_title,descrption
0,2kyS6SvSYSE,2017-11-14,22,2017-11-13 17:13:01,748374,57527,2966,15954,People & Blogs,shantell martin,we want to talk about our marriage,caseyneistat,shantell s channel https www youtube com shant...
1,1ZAPwfrtAFY,2017-11-14,24,2017-11-13 07:30:00,2418783,97185,6146,12703,Entertainment,last week tonight trump presidency last week t...,the trump presidency last week tonight with jo...,lastweektonight,one year after the presidential election john ...


In [131]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40949 entries, 0 to 40948
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   video_id       40949 non-null  object        
 1   trending_date  40949 non-null  datetime64[ns]
 2   category_id    40949 non-null  int64         
 3   publish_time   40949 non-null  datetime64[ns]
 4   views          40949 non-null  int64         
 5   likes          40949 non-null  int64         
 6   dislikes       40949 non-null  int64         
 7   comment_count  40949 non-null  int64         
 8   category_name  40949 non-null  category      
 9   tags           40949 non-null  object        
 10  video_title    40949 non-null  object        
 11  channel_title  40949 non-null  object        
 12  descrption     40949 non-null  object        
dtypes: category(1), datetime64[ns](2), int64(5), object(5)
memory usage: 5.1+ MB


Hasta este punto parece que tenemos un Dataset limpio y listo para hacer nuestros primeros analisis, es momento de exportarlo, para ser usado cuando lo necesitemos.


In [134]:
df.to_csv('../data/preprocessed/video_cln.csv', index=False)

In [132]:

# print("""
#       ****************************************
#       TOTAL DE DATOS DUPLICADOS EN EL DATASET
#       ****************************************""")
# print(df.duplicated().sum())